In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "/share/dutta/eyao/dataset/kaggle/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/share/dutta/eyao/dataset/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory () that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Importing Data**

In [ ]:
train = pd.read_csv('/share/dutta/eyao/dataset/kaggle/house-prices-advanced-regression-techniques/train.csv')
test = pd.read_csv('/share/dutta/eyao/dataset/kaggle/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
train.head()

In [ ]:
train.info()

# Summary about data

In [ ]:
train.shape

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

# Data Cleaning

In [ ]:
def fill_missing_values(df):
    for column in df.columns:
        if df[column].dtype == np.float64:
            df[column].fillna(df[column].mean(), inplace=True)
        elif (df[column] == 'O').any():
            df[column].fillna(df[column].mode().iloc[0], inplace=True)

In [ ]:
fill_missing_values(train)

In [ ]:
train.isnull().sum()

In [ ]:
fill_missing_values(test)

In [ ]:
# columns_to_drop = ['Id']
# train = train.drop(columns=columns_to_drop)

# Encoding Data

In [ ]:
from sklearn.preprocessing import LabelEncoder

def encode_categorical_columns(df):
    label_encoder = LabelEncoder()

    for column in df.columns:
        if df[column].dtype == 'object':
            df[column] = label_encoder.fit_transform(df[column])
        elif df[column].dtype == 'float64':
            df[column] = df[column].fillna(df[column].mean())

    return df

In [ ]:
train = encode_categorical_columns(train)
test = encode_categorical_columns(test)

In [ ]:
# display_column_data_types(train)

In [ ]:
X = train.drop(['SalePrice','Id'], axis=1)
y = train['SalePrice']
X_val = test.drop(['Id'],axis = 1)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)
X_val = sc.fit_transform(X_val)

# Training Data

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# def encode_labels(y):
#     label_encoder = LabelEncoder()
#     y_encoded = label_encoder.fit_transform(y)
#     return y_encoded

In [ ]:
# y_train_encoded = encode_labels(y_train)
# y_test_encoded = encode_labels(y_test)

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import  accuracy_score


In [ ]:
# seed = np.random.seed(0)
# from sklearn.ensemble import RandomForestRegressor
# rfmodel = RandomForestRegressor(n_estimators = 1000,random_state = 0,criterion="absolute_error")
# rfmodel.fit(X_train,y_train)
# print("MSE of Random Forest Regressor: ",mean_square_error(y_test,rfmodel.predict(X_test)))

# Training Models to find best fit model

first is the XGBRegressor

https://www.geeksforgeeks.org/xgboost-for-regression/

In [ ]:
from xgboost import XGBRegressor

xgb_regressor = XGBRegressor(random_state=0,learning_rate=0.5, max_depth=5, n_estimators=400)
xgb_regressor.fit(X_train, y_train)

In [ ]:
y_test.head()

In [ ]:
print(f"Mean Squared Error: {mean_squared_error(y_test, xgb_regressor.predict(X_test))}")

using LGBMRegressor
https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.LGBMRegressor.html

In [ ]:
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error
lgbmodel = LGBMRegressor(random_state=0,max_depth= 8,n_estimators= 5000)
lgbmodel.fit(X_train,y_train)
print("MSE of Light Gradient Boosted Machine Regressor: ",mean_squared_error(y_test,lgbmodel.predict(X_test)))

using CatBoostRegressor

https://catboost.ai/en/docs/concepts/python-usages-examples

In [ ]:
from catboost import CatBoostRegressor
catmodel = CatBoostRegressor(random_state = 0,loss_function="MAE",verbose=False)
catmodel.fit(X_train,y_train)
print("MSE of Category Boosting Regressor: ",mean_squared_error(y_test,catmodel.predict(X_test)))

# HyperParameter Tuning

HyperParameter Tuning using RandomizedSearchCv (GridSearchCv is very computionally extensive)

https://www.kdnuggets.com/hyperparameter-tuning-gridsearchcv-and-randomizedsearchcv-explained

In [ ]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
random_forest = RandomForestRegressor(random_state=0)
distributions = {
    'n_estimators': randint(low=10, high=200),
    'max_depth': randint(low=1, high=20),
    'min_samples_split': randint(low=2, high=20),
    'min_samples_leaf': randint(low=1, high=20),
}
rf_search = RandomizedSearchCV(random_forest, distributions, random_state=0)
rf_search.fit(X_train, y_train)

In [ ]:
rf_search.best_params_

In [ ]:
best_params = rf_search.best_params_
best_rf_model = RandomForestRegressor(random_state=0, **best_params)
best_rf_model.fit(X_train, y_train)
y_pred = best_rf_model.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

least error is through XGBRegressor
hence best model

# Submission File

In [ ]:
submission = pd.read_csv('/share/dutta/eyao/dataset/kaggle/house-prices-advanced-regression-techniques/sample_submission.csv')

In [ ]:
submission.head()

In [ ]:
# test_without_id = test.drop('Id', axis=1)
predictions = xgb_regressor.predict(X_val)

In [ ]:
# submission['SalePrice'] = predictions

submission = pd.DataFrame({'Id': test.Id ,'SalePrice': predictions})

submission.to_csv('submission.csv', index=False)

print(submission.head())